In [1]:
%%time
%run prepare_variables.py

CPU times: user 9.18 s, sys: 404 ms, total: 9.58 s
Wall time: 9.64 s


In [2]:
p.shape

(7364, 429429)

In [3]:
p

<7364x429429 sparse matrix of type '<class 'numpy.float64'>'
	with 1581374 stored elements in COOrdinate format>

#### That is very sparse - only 0.05% of the matrix has anything in it.

#### http://scikit-learn.org/stable/auto_examples/text/document_clustering.html#sphx-glr-auto-examples-text-document-clustering-py

#### http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.Normalizer.html#sklearn.preprocessing.Normalizer

In [2]:
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
normalizer = Normalizer(copy=False)
from sklearn.cluster import KMeans
from numpy.random import random

In [8]:
%%time
svd = TruncatedSVD(n_components=1000, n_iter=7, random_state=42)
pipeline = make_pipeline(svd, normalizer)
p_new1 = pipeline.fit_transform(p)
p_new1.shape

CPU times: user 10min 11s, sys: 1min 31s, total: 11min 43s
Wall time: 5min 8s


In [9]:
%%time
svd = TruncatedSVD(n_components=2000, n_iter=7, random_state=42)
pipeline = make_pipeline(svd, normalizer)
p_new2 = pipeline.fit_transform(p)

CPU times: user 30min 52s, sys: 1min 35s, total: 32min 28s
Wall time: 12min 24s


In [10]:
%%time
svd = TruncatedSVD(n_components=150, n_iter=100, random_state=42)
pipeline = make_pipeline(svd, normalizer)
p_new3 = pipeline.fit_transform(p)

CPU times: user 8min, sys: 3min 17s, total: 11min 18s
Wall time: 6min 9s


In [3]:
%%time
p_new4 = normalizer.fit_transform(p)

CPU times: user 364 ms, sys: 16 ms, total: 380 ms
Wall time: 379 ms


In [5]:
#  p_new = normalizer.fit_transform(p)

In [35]:
print (p_new1.shape, p_new2.shape, p_new3.shape, p_new4.shape)

(7364, 1000) (7364, 2000) (7364, 150) (7364, 429429)


In [11]:
%%time
kmeans1 = KMeans(n_clusters=5, n_jobs=-1).fit(p_new1) 

CPU times: user 1.05 s, sys: 80 ms, total: 1.13 s
Wall time: 3.79 s


In [12]:
%%time
kmeans2 = KMeans(n_clusters=5, n_jobs=-1).fit(p_new2) 

CPU times: user 2.03 s, sys: 100 ms, total: 2.13 s
Wall time: 6.65 s


In [13]:
%%time
kmeans3 = KMeans(n_clusters=5, n_jobs=-1).fit(p_new3) 

CPU times: user 196 ms, sys: 44 ms, total: 240 ms
Wall time: 800 ms


In [4]:
%%time
kmeans4 = KMeans(n_clusters=5, n_jobs=-1).fit(p_new4)

CPU times: user 604 ms, sys: 152 ms, total: 756 ms
Wall time: 5min 28s


In [15]:
type(p_new1)

numpy.ndarray

In [16]:
type(kmeans1)

sklearn.cluster.k_means_.KMeans

In [27]:
np.save('p_new1', p_new1)
np.save('p_new2', p_new2)
np.save('p_new3', p_new3)
np.save('p_new4', p_new4)
np.save('kmeans1', kmeans1)
np.save('kmeans2', kmeans2)
np.save('kmeans3', kmeans3)
np.save('kmeans4', kmeans4)

## How many videos has it found in each cluster?

In [24]:
kmeans = kmeans2

In [25]:
videos_df['cluster_labels'] = kmeans.labels_
videos_df.groupby('cluster_labels').count()[['videoTitle']]

,videoTitle
cluster_labels,
0,818
1,1649
2,290
3,2124
4,2483


## Sample ten videos randomly from each of those groups:

In [26]:
pd.set_option('display.max_rows', 120)

# https://stackoverflow.com/questions/22472213/python-random-selection-per-group

size = 10        # sample size
replace = False  # with replacement
fn = lambda obj: obj.loc[np.random.choice(obj.index, size, replace),:]
clustered_sample = videos_df.groupby('cluster_labels', as_index=False).apply(fn)
clustered_sample[['videoTitle','cluster_labels', 'link']]

videoTitle  cluster_labels  \
0 4393                             Cooking With Fernando!               0   
  4410                               how to make mutanjan               0   
  4089  Tsubako cooking chili herb dip　簡単ツバコクッキング　チリハー...               0   
  1447  How To: Simple Egg Masala Fry | Ventuno Home C...               0   
  7356  Đồ Chơi Nấu Ăn Disney Princess Kitchen Toy Coo...               0   
  6665  Low-Carb Shirataki Pasta with Meat Sauce Recip...               0   
  2154  Gongura chicken curry - How to cook Chicken cu...               0   
  6493                  What's Cooking: Sweet Potato Hash               0   
  1498  Aloo capsicum fry - How to cook potato capsicu...               0   
  6628  Cooking Goat Blood with Goat Meat in Our Villa...               0   
1 201                  Cooking 101: Baked Chicken Nuggets               1   
  701             Alvita Cooking watch out for that knife               1   
  1463                        Big Game Rib Cook Off Pt. 2               1   
  349   Haute Cuisine | On Call with the Prairie Doc |...               1   
  1917       Cooking with the Sassy Chef – Winter Warmers               1   
  3452                          MAMMA VS PAPA' IN CUCINA!               1   
  1379                    Thoughts You Have While Cooking               1   
  3588  HSN | The Monday Night Show with Adam Freeman ...               1   
  4432  HSN | HSN Today: Holiday Cooking featuring Das...               1   
  4984  WSRE | Gourmet Cooking with Earl Peyroux | Epi...               1   
2 205                                   Upward Facing Dog               2   
  6611              Highly Effective Dog Training Secrets               2   
  46                  10 Most DANGEROUS DOGS in the World               2   
  5943             The Best Time Of Day To Train Your Dog               2   
  3225  Willard nonprofit trains then donates police d...               2   
  4456  Teach Your Dog to Listen - Use Reward Scaling ...               2   
  2349  Puppy Training: Games For Teaching Come When C...               2   
  2682        How to Make Sooo Cali Dogs |  Hilah Cooking               2   
  2539                              What is dog training?               2   
  3891  Rob Peladeau: "Behaviorist Dog Trainer" | Talk...               2   
3 6435  FutureGrid MOOC: IPOP Unit 6: IPOP Architectur...               3   
  5590   Leveraging Oracle's Life Sciences Data Hub (LSH)               3   
  678   IU Cloud Computing MOOC: Virtue of Virtual 3: ...               3   
  6261  [PL] Machine learning linear regression. Imple...               3   
  4483        Data in JSON Format - Intro to Data Science               3   
  5546                          2. Large Margin Intuition               3   
  3940        Northwestern Cardiologist Elizabeth McNally               3   
  2191                           Cooking One Handed video               3   
  905                     3. Learning Sigmoid Belief Nets               3   
  461                                    4. Plotting Data               3   
4 975   Attic to Basement Estate Cleanouts Episode 54 ...               4   
  766   Excel Data Analysis - Move Data From Column To...               4   
  5845                           Culinary Specialist (CS)               4   
  5565                             Computing sharpe ratio               4   
  1150  Cooking Mama,Cooking Mama 2,Cooking Mama Colle...               4   
  1548  Ginger Garlic Paste, Dry Coconut Paste - Vatan...               4   
  6234          Torch - Ep. 19 (Deep Learning SIMPLIFIED)               4   
  6400               Sights & sounds of cooking at Scenns               4   
  5129  ইংলিশ ব্রেকফাস্ট ||| Restaurant Style English ...               4   
  5529  Broken Arrow police aim to train dog walkers t...               4   

                                               link  
0 4393  https://www.youtube.com/watch?v=mwPxfE-PUb8  
  4410  htt